In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
import time
import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

url = input('Give me eplease the URL')
tname = input('print the tag name')
a = 'Prwto_Thema'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
date_list = []
tag_list = []

stop_date = 20230731
scroll_pause_time = 2.6

#       Poll
driver.get(url)

time.sleep(2)
driver.maximize_window()
time.sleep(1)
disagree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

flag = True
inside_flag = True
def scroll(driver, timeout):
    global flag
    global stop_date
    global inside_flag
    global tag_list
    global date_list
    scroll_pause_time = timeout
    # its the nr of scrollings that after that i check the date.
    nr_of_scrollings = 2
    changable_nr_of_scrollings = nr_of_scrollings
    #inside_flag = True

    def date_check_fun():
        global inside_flag

        # In time, when the function gets inside the main function the coment section here(2 lines)
        # will need to replace the 2 bellow them.
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'lxml')
        articles_place = soup.find('div' , class_ = "listLoop")
        dates = articles_place.find_all('div' , class_ = "wrp")
        for i in dates:
            date = i.find('time')
            date = date.get_text()
            date = date.split(",")
            date = date[0].split(".")
            date.reverse()
            date = "".join(date)
            date = int(date)
            #print(date)

        if date < stop_date :
                print('the date is smaller. Programm Must STOP')
                inside_flag = False
                return inside_flag
    

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while inside_flag:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            #flag=False
            print('the page reaches the end for the first time. It cannot scroll anymore')
            break
        last_height = new_height
        time.sleep(scroll_pause_time)
        # i want every some scroll downs to check the date
        changable_nr_of_scrollings = changable_nr_of_scrollings - 1
        #nr_of_scrollings = nr_of_scrollings - 1
        if changable_nr_of_scrollings == 0:
            date_check_fun()
            #print('on the outer/mother function the inside_flag is  ' , inside_flag)
            #nr_of_scrollings = 2 
            changable_nr_of_scrollings = nr_of_scrollings

    #   THIS PART OF CODE I WORK now. The apo epanw functions properly.
    
    def tag_fun(inside):
        global tag_list
        inside_tag_list = []

        from bs4 import BeautifulSoup
        import requests

        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')
        tags = soup.find('div' , class_ = "tagsCnt")
        tags = tags.find_all('a')
        for a in tags:
            tag = a.get_text()
            tag = tag.replace('\r' , "")
            tag = tag.replace('\n' ,  "")
            tag = tag.strip()
            inside_tag_list.append(tag)   
        tag_list.append(inside_tag_list)
    

    #inside_flag = True
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')

    articles_place = soup.find('div' , class_ = "listLoop")
    articles = soup.find_all('article')

    for i in articles:
        date = i.find('div' , class_ = "wrp")
        date = date.find('time')
        date = date.get_text()
        date = date.split(",")
        date = date[0].split(".")
        date.reverse()
        date = "".join(date)
        date = int(date)
        #print(date)

        if date < stop_date :
            print('the date is smaller. I need to terminate the whole programm')
            break
        else:
            article_url = i.find('div' , class_ = 'heading')
            article_url = article_url.find('a')
            article_url = article_url.get('href')
            date_list.append(date)
            #print(article_url)
            https = "https://www.protothema.gr"
            if article_url.find(https)<0:
                article_url = https + article_url
                print('apo katw article url ' , article_url)
            
            tag_fun(article_url)
        

scroll(driver , scroll_pause_time)

print(tag_list)
print(len(tag_list))
print(len(date_list))
print('the last date is ' , date_list[-1])

import pandas as pd
# df_costliness = pd.DataFrame(tag_list,date_list)
# df_costliness.to_csv('costliness.csv')

#print(tag_list)
#print(date_list)

new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        #if tag_list[i][j] not in new_tag_list:
       new_tag_list.append(tag_list[i][j])
#print(new_tag_list)

res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

resultList = new_list = list(map(list, res.items()))
# printing the resultant list of a dictionary key-values
print(resultList)

df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)


# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=date_list
ttag_list = tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)

n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    for tag in i:
        inside=[]
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)


print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))

the page has reached its end. It cannot scroll anymore


KeyboardInterrupt: 

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
import time
import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

url = input('Give me eplease the URL')
tname = input('print the tag name')
a = 'Prwto_Thema'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
date_list = []
tag_list = []

stop_date = 20230731
scroll_pause_time = 1.8

#       Poll
driver.get(url)

time.sleep(2)
driver.maximize_window()
time.sleep(1)
disagree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

flag = True
inside_flag = True
def scroll(driver, timeout):
    global flag
    global stop_date
    global inside_flag
    global tag_list
    global date_list
    scroll_pause_time = timeout
    # its the nr of scrollings that after that i check the date.
    nr_of_scrollings = 2
    changable_nr_of_scrollings = nr_of_scrollings
    #inside_flag = True

    def date_check_fun():
        global inside_flag

        # In time, when the function gets inside the main function the coment section here(2 lines)
        # will need to replace the 2 bellow them.
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'lxml')
        articles_place = soup.find('div' , class_ = "listLoop")
        dates = articles_place.find_all('div' , class_ = "wrp")
        for i in dates:
            date = i.find('time')
            date = date.get_text()
            date = date.split(",")
            date = date[0].split(".")
            date.reverse()
            date = "".join(date)
            date = int(date)
            #print(date)

        if date < stop_date :
                print('the date is smaller. Programm Must STOP')
                inside_flag = False
                return inside_flag
    

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while inside_flag:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            #flag=False
            time.sleep(scroll_pause_time)
# second try
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(scroll_pause_time)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height==last_height:
                print('the page reaches the end for the first time. It cannot scroll anymore')
                break
            last_height=new_height
# second try
        last_height = new_height
        time.sleep(scroll_pause_time)
        # i want every some scroll downs to check the date
        changable_nr_of_scrollings = changable_nr_of_scrollings - 1
        #nr_of_scrollings = nr_of_scrollings - 1
        if changable_nr_of_scrollings == 0:
            date_check_fun()
            #print('on the outer/mother function the inside_flag is  ' , inside_flag)
            #nr_of_scrollings = 2 
            changable_nr_of_scrollings = nr_of_scrollings

    #   THIS PART OF CODE I WORK now. The apo epanw functions properly.
    
    def tag_fun(inside):
        global tag_list
        inside_tag_list = []

        from bs4 import BeautifulSoup
        import requests

        page = requests.get(inside)
        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            tags = soup.find('div' , class_ = "tagsCnt")
            tags = tags.find_all('a')
            for a in tags:
                tag = a.get_text()
                tag = tag.replace('\r' , "")
                tag = tag.replace('\n' ,  "")
                tag = tag.strip()
                inside_tag_list.append(tag)   
            tag_list.append(inside_tag_list)
        except:
            print('Propably  have found a brokenarticle. Its url is ' , inside)

    #inside_flag = True
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')

    articles_place = soup.find('div' , class_ = "listLoop")
    articles = soup.find_all('article')

    for i in articles:
        date = i.find('div' , class_ = "wrp")
        date = date.find('time')
        date = date.get_text()
        date = date.split(",")
        date = date[0].split(".")
        date.reverse()
        date = "".join(date)
        date = int(date)
        #print(date)

        if date < stop_date :
            print('the date is smaller. I need to terminate the whole programm')
            break
        else:
            article_url = i.find('div' , class_ = 'heading')
            article_url = article_url.find('a')
            article_url = article_url.get('href')
            date_list.append(date)
            #print(article_url)
            https = "https://www.protothema.gr"
            if article_url.find(https)<0:
                print('xlasmeno article url ' , article_url)
            
            tag_fun(article_url)
        

scroll(driver , scroll_pause_time)

print(tag_list)
print(len(tag_list))
print(len(date_list))
print('the last date is ' , date_list[-1])

import pandas as pd
# df_costliness = pd.DataFrame(tag_list,date_list)
# df_costliness.to_csv('costliness.csv')

#print(tag_list)
#print(date_list)

new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        #if tag_list[i][j] not in new_tag_list:
       new_tag_list.append(tag_list[i][j])
#print(new_tag_list)

res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

resultList = new_list = list(map(list, res.items()))
# printing the resultant list of a dictionary key-values
print(resultList)

df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)


# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=date_list
ttag_list = tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)

n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    for tag in i:
        inside=[]
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)


print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))

the date is smaller. Programm Must STOP
apo katw article url  https://www.protothema.grhttps://www.ygeiamou.gr/%ce%b5%ce%b9%ce%b4%ce%ae%cf%83%ce%b5%ce%b9%cf%82/%cf%80%ce%bf%ce%bb%ce%b9%cf%84%ce%b9%ce%ba%ce%ae-%cf%85%ce%b3%ce%b5%ce%af%ce%b1%cf%82/393226/techniti-noimosini-o-protagonistis-tou-5ou-sinedriou-ygeiamou/


ConnectionError: HTTPSConnectionPool(host='www.protothema.grhttps', port=443): Max retries exceeded with url: //www.ygeiamou.gr/%CE%B5%CE%B9%CE%B4%CE%AE%CF%83%CE%B5%CE%B9%CF%82/%CF%80%CE%BF%CE%BB%CE%B9%CF%84%CE%B9%CE%BA%CE%AE-%CF%85%CE%B3%CE%B5%CE%AF%CE%B1%CF%82/393226/techniti-noimosini-o-protagonistis-tou-5ou-sinedriou-ygeiamou/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E7790E2F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))